# Drone Model-Predictive Control

## Package Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder,DirectCollocation, FindResourceOrThrow, 
    MathematicalProgram, MeshcatVisualizerCpp, Parser, plot_system_graphviz, PiecewisePolynomial,   
     SceneGraph, Simulator, Solve, StartMeshcat, TrajectorySource, VectorSystem )

from pydrake.examples.quadrotor import (QuadrotorGeometry, QuadrotorPlant, StabilizingLQRController)

from underactuated.meshcat_cpp_utils import MeshcatSliders



In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://7a2da9b8-cd38-4703-b181-4b0e0b197067.deepnoteproject.com/7000/


## Trajectory Optimization for the Quadrotor

Find an optimal trajectory from an initial condition to a target destination, that minimizes time to target destination, and adheres to thrust limits

In [5]:
def optimize_quadrotor_trajectory(quadrotor_plant, quadrotor_context, init_state, final_state, thrust_limit, num_time_samples):

    # use direct collocation to integrate quadrotor dynamics
    # DirectCollation classe performs Direct transcription trajectory optimization using nonlinear programming and collocation
    # It assumes a first-order hold on the input trajectory and a cubic spline representation of the state trajectory, and adds dynamic constraints 
    # (and running costs) to the midpoints as well as the breakpoints in order to achieve a 3rd order integration accuracy.
    dircol = DirectCollocation(quadrotor_plant,
                            quadrotor_context,
                            num_time_samples=num_time_samples,
                            minimum_timestep=0.05,
                            maximum_timestep=0.2)

    #Returns a reference to the MathematicalProgram associated with the trajectory optimization problem.
    prog = dircol.prog()

    # add constraint that enforces that all timesteps have equal duration (ensures more efficient optimization)
    dircol.AddEqualTimeIntervalsConstraints()

    # Returns placeholder decision variables (not actually declared as decision variables in the MathematicalProgram) 
    # associated with the input, u, but with the time-index undetermined.
    u = dircol.input()

    print(f"input = {u}")

    # enforce individual thrust limits for each propellor
    dircol.AddConstraintToAllKnotPoints(u[0] <= thrust_limit)
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[0])

    dircol.AddConstraintToAllKnotPoints(u[1] <= thrust_limit)
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[1])

    dircol.AddConstraintToAllKnotPoints(u[2] <= thrust_limit)
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[2])

    dircol.AddConstraintToAllKnotPoints(u[3] <= thrust_limit)
    dircol.AddConstraintToAllKnotPoints(-thrust_limit <= u[3])

    # add initial state constraint (first parameter is upper bound, second parameter is lower bound, 3rd parameter is variable being constrained)
    prog.AddBoundingBoxConstraint(init_state, init_state,
                                  dircol.initial_state())

    # add final state constraint
    prog.AddBoundingBoxConstraint(final_state, final_state,
                                  dircol.final_state())


    R = 100  # Cost on input "effort".
    dircol.AddRunningCost(R * u[0]**2)
    dircol.AddRunningCost(R * u[1]**2)
    dircol.AddRunningCost(R * u[2]**2)
    dircol.AddRunningCost(R * u[3]**2)

    # Add a final cost equal to the total duration.
    dircol.AddFinalCost(dircol.time())

    # Construct an initial guess at an optimal trajectory
    # PiecewisePolynomial.FirstOrderHold Constructs a piecewise linear PiecewisePolynomial using matrix samples.
    breaks = [0., 4.]
    samples = np.column_stack((initial_state, final_state))
    initial_x_trajectory = PiecewisePolynomial.FirstOrderHold(breaks, samples)  # yapf: disable

    dircol.SetInitialTrajectory(PiecewisePolynomial(), initial_x_trajectory)

    result = Solve(prog)
    assert (result.is_success())

    u_trajectory = dircol.ReconstructInputTrajectory(result)
    x_trajectory = dircol.ReconstructStateTrajectory(result)

    return u_trajectory, x_trajectory


#calculate optimal trajectory and simulation

initial_state = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

final_state = [-1, -3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# limit is the 2-norm of the thrust vector 
thrust_limit = 20.0
#we want to have an odd number
num_time_samples = 21

quadrotor_plant = QuadrotorPlant()
quadrotor_context = quadrotor_plant.CreateDefaultContext()

u_trajectory, x_trajectory = optimize_quadrotor_trajectory(quadrotor_plant, quadrotor_context, initial_state, final_state, thrust_limit, num_time_samples)

builder = DiagramBuilder()
source = builder.AddSystem(TrajectorySource(x_trajectory))

# Set up visualization in MeshCat
scene_graph = builder.AddSystem(SceneGraph())
QuadrotorGeometry.AddToBuilder(builder, source.get_output_port(0), scene_graph)
meshcat.Delete()
meshcat.ResetRenderMode()
meshcat.SetProperty('/Background','visible',False)
MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
# end setup for visualization

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
sim_context = simulator.get_mutable_context()

#sim_context.SetContinuousState(initial_state)

simulator.set_target_realtime_rate(1.0)
meshcat.AddButton('Stop Simulation')
while meshcat.GetButtonClicks('Stop Simulation') < 1:
    
    simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)

meshcat.DeleteAddedControls()

    


input = [Variable('u(0)', Continuous) Variable('u(1)', Continuous)
 Variable('u(2)', Continuous) Variable('u(3)', Continuous)]


## Wire Up Block Diagram &amp; Run simulation

In [ ]:
def simulate_quadrotor():

    #define diagram builder, used to connect plant and controller inputs / outputs
    builder = DiagramBuilder()


    quadrotor_plant = QuadrotorPlant()

    plant = builder.AddSystem(quadrotor_plant)

    controller = builder.AddSystem(StabilizingLQRController(plant, [1, 2, 2]))
    builder.Connect(controller.get_output_port(0), plant.get_input_port(0))
    builder.Connect(plant.get_output_port(0), controller.get_input_port(0))

    # Set up visualization in MeshCat
    scene_graph = builder.AddSystem(SceneGraph())
    QuadrotorGeometry.AddToBuilder(builder, plant.get_output_port(0), scene_graph)
    meshcat.Delete()
    meshcat.ResetRenderMode()
    meshcat.SetProperty('/Background','visible',False)
    MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    # end setup for visualization



    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()



    #continuous_state = context.get_mutable_continuous_state_vector()

    # Set the initial conditions (theta1, theta2, theta1dot, theta2dot)
    #context.SetContinuousState([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    context.SetContinuousState(0.5*np.random.randn(12,))

   # print(f"continuous state {continuous_state}")
    #print(f"size of continuous state {continuous_state.size()}")
    simulator.set_target_realtime_rate(1.0)
    meshcat.AddButton('Stop Simulation')
    while meshcat.GetButtonClicks('Stop Simulation') < 1:
        
        simulator.AdvanceTo(simulator.get_context().get_time() + 1.0)

    meshcat.DeleteAddedControls()

    print("simulation complete")

#simulate_quadrotor()

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7a2da9b8-cd38-4703-b181-4b0e0b197067' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>